# Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
import os

fake = Faker()
random.seed(42)
Faker.seed(42)

# Create output folder
os.makedirs("finance_data", exist_ok=True)

# Helper Functions

In [3]:
def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + timedelta(days=random.randint(0, (end - start).days))

def dirty_string(s):
    """Introduce casing and whitespace issues"""
    if random.random() < 0.3:
        s = s.upper()
    elif random.random() < 0.6:
        s = s.lower()
    return s.strip() + (" " if random.random() < 0.1 else "")


# Generate customers Table

In [5]:
customers = []
for i in range(1000):
    name = fake.name()
    email = fake.email() if random.random() > 0.1 else None
    phone = fake.phone_number() if random.random() > 0.1 else None
    dob = fake.date_of_birth(minimum_age=18, maximum_age=70)
    if random.random() < 0.03:
        dob = datetime.now() + timedelta(days=random.randint(1, 100))  # Future DOB
    location = dirty_string(fake.city())
    occupation = fake.job()
    customers.append([i+1, dirty_string(name), email, phone, dob, fake.date_between(start_date='-10y', end_date='-1d'), location, occupation])

customers_df = pd.DataFrame(customers, columns=[
    "customer_id", "name", "email", "phone", "date_of_birth", "account_open_date", "location", "occupation"
])

# Add 5% duplicates
customers_df = pd.concat([customers_df, customers_df.sample(frac=0.05, random_state=1)], ignore_index=True)

customers_df.head()

print(f"Generated customers table with {len(customers_df)} rows.")


Generated customers table with 1050 rows.


# Generate accounts Table

In [7]:
account_types = ["savings", "checking", "business"]

accounts = []
for i in range(1200):
    acc_id = 10000 + i
    customer_id = random.choice(customers_df["customer_id"].tolist() + [2000])  # Invalid FK
    acc_type = random.choice(account_types)
    balance = round(random.uniform(-5000, 100000), 2)
    status = random.choice(["active", "closed", "frozen"])
    last_updated = fake.date_between(start_date='-1y', end_date='today')
    accounts.append([acc_id, customer_id, acc_type, balance, status, last_updated])

accounts_df = pd.DataFrame(accounts, columns=[
    "account_id", "customer_id", "account_type", "balance", "status", "last_updated"
])

# Add 2% duplicates
accounts_df = pd.concat([accounts_df, accounts_df.sample(frac=0.02, random_state=2)], ignore_index=True)

accounts_df.head()

print(f"Generated accounts table with {len(accounts_df)} rows.")


Generated accounts table with 1224 rows.


# Generate transactions Table

In [10]:
categories = ["groceries", "shopping", "fuel", "entertainment", "utilities", "salary", "insurance", "rent"]
category_typos = ["Grocerry", "Groceriess", "SHOPPING", "Fuel ", "Entertaiment"]

transactions = []
for i in range(25000):
    tx_id = 200000 + i
    acc_id = random.choice(accounts_df["account_id"].tolist())
    tx_date = fake.date_between(start_date='-1y', end_date='today')
    if random.random() < 0.02:
        tx_date = datetime.now() + timedelta(days=random.randint(1, 30))  # Future date
    tx_type = random.choice(["credit", "debit"])
    amount = round(random.uniform(1, 10000), 2)
    if tx_type == "credit" and random.random() < 0.1:
        amount = -amount  # Messy: negative credit
    merchant = fake.company() if random.random() > 0.08 else None
    category = random.choice(categories + category_typos)
    transactions.append([tx_id, acc_id, tx_date, tx_type, amount, merchant, category])

transactions_df = pd.DataFrame(transactions, columns=[
    "transaction_id", "account_id", "transaction_date", "transaction_type", "amount", "merchant", "category"
])

transactions_df.head()

print(f"Generated transactions table with {len(transactions_df)} rows.")


Generated transactions table with 25000 rows.


# Generate loan_applications Table

In [12]:
statuses = ["approved", "rejected", "pending", "APPROVED", "Approved ", " REJECTED"]

loans = []
for i in range(2000):
    loan_id = 300000 + i
    cust_id = random.choice(customers_df["customer_id"].tolist())
    loan_type = random.choice(["personal", "home", "car", "education"])
    amount_applied = round(random.uniform(0, 500000), 2)
    if random.random() < 0.05:
        amount_applied = None
    status = random.choice(statuses)
    application_date = fake.date_between(start_date='-2y', end_date='-1d')
    approved_date = application_date + timedelta(days=random.randint(-10, 30))  # May be wrong
    loans.append([loan_id, cust_id, loan_type, amount_applied, status, application_date, approved_date])

loans_df = pd.DataFrame(loans, columns=[
    "loan_id", "customer_id", "loan_type", "amount_applied", "status", "application_date", "approved_date"
])

loans_df.head()

print(f"Generated loan_applications table with {len(loans_df)} rows.")


Generated loan_applications table with 2000 rows.


#  Generate credit_scores Table

In [14]:
scores = []
for i in range(5000):
    cust_id = random.choice(customers_df["customer_id"].tolist())
    score_date = fake.date_between(start_date='-2y', end_date='today')
    score = random.randint(250, 950)
    if random.random() < 0.05:
        score = None
    scores.append([cust_id, score, score_date])

credit_df = pd.DataFrame(scores, columns=["customer_id", "score", "score_date"])

# Add 5% duplicates
credit_df = pd.concat([credit_df, credit_df.sample(frac=0.05, random_state=3)], ignore_index=True)

credit_df.head()

print(f"Generated credit_scores table with {len(credit_df)} rows.")


Generated credit_scores table with 5250 rows.


In [16]:
customers_df.to_csv("finance_data/customers.csv", index=False)
accounts_df.to_csv("finance_data/accounts.csv", index=False)
transactions_df.to_csv("finance_data/transactions.csv", index=False)
loans_df.to_csv("finance_data/loan_applications.csv", index=False)
credit_df.to_csv("finance_data/credit_scores.csv", index=False)

print("All messy data saved to 'finance_data/' folder.")


All messy data saved to 'finance_data/' folder.


# Fix loan_application csv

In [20]:
import pandas as pd

# Load the original file
df = pd.read_csv("finance_data/loan_applications.csv")
print(f" Original rows: {df.shape[0]}")
df.head()


 Original rows: 2000


,loan_id,customer_id,loan_type,amount_applied,status,application_date,approved_date
0,300000,525,home,383056.57,REJECTED,2024-05-22,2024-05-15
1,300001,169,personal,124109.38,pending,2024-09-16,2024-09-24
2,300002,485,personal,335275.13,pending,2024-08-01,2024-08-23
3,300003,801,car,167905.24,Approved,2024-10-15,2024-10-08
4,300004,251,home,39309.12,APPROVED,2024-02-01,2024-02-24


In [22]:
# Replace NaNs with the string 'NULL'
df = df.fillna('NULL')

# Optional: Check how many were replaced
print("Replaced missing values with 'NULL'")
df.head()


Replaced missing values with 'NULL'


,loan_id,customer_id,loan_type,amount_applied,status,application_date,approved_date
0,300000,525,home,383056.57,REJECTED,2024-05-22,2024-05-15
1,300001,169,personal,124109.38,pending,2024-09-16,2024-09-24
2,300002,485,personal,335275.13,pending,2024-08-01,2024-08-23
3,300003,801,car,167905.24,Approved,2024-10-15,2024-10-08
4,300004,251,home,39309.12,APPROVED,2024-02-01,2024-02-24


In [24]:
# Export to a new cleaned file
df.to_csv("finance_data/loan_applications_clean.csv", index=False)
print("Cleaned file saved as 'loan_applications_clean.csv'")


Cleaned file saved as 'loan_applications_clean.csv'


# Fix credit_score csv

In [26]:
import pandas as pd

# Load the existing credit scores CSV
df = pd.read_csv("finance_data/credit_scores.csv")

print(f"Original rows: {df.shape[0]}")
df.head()



Original rows: 5250


,customer_id,score,score_date
0,333,511.0,2023-08-26
1,497,823.0,2023-12-17
2,597,480.0,2023-11-19
3,647,550.0,2024-11-12
4,687,527.0,2024-10-04


In [28]:
# Replace all NaN (empty) values with the string 'NULL'
df.fillna("NULL", inplace=True)

print(" Replaced all missing values with 'NULL'")
df.head()


 Replaced all missing values with 'NULL'


C:\Users\DELL.WIN-EFQG1MH1O15\AppData\Local\Temp\ipykernel_13500\3193140060.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NULL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("NULL", inplace=True)


,customer_id,score,score_date
0,333,511.0,2023-08-26
1,497,823.0,2023-12-17
2,597,480.0,2023-11-19
3,647,550.0,2024-11-12
4,687,527.0,2024-10-04


In [30]:
# Save cleaned version
df.to_csv("finance_data/credit_scores_clean.csv", index=False)
print(" Saved cleaned file as 'credit_scores_clean.csv'")


 Saved cleaned file as 'credit_scores_clean.csv'


In [32]:
# Check if any missing values remain
print(" Any remaining blanks:", df.isnull().sum().sum())  # Should be 0


 Any remaining blanks: 0
